# 반복 실험을 위한 Sacred
- Sacred란?
    - Sacred is a tool to help you configure, organize, log and reproduce experiments developed at IDSIA
    - 머신러닝 모델링을 진행할 때 설정을 저장해주고 관리하는 것을 도와주는 도구
- 왜 필요한가요?
    - Kaggle에서 자주 발생하는 예시
        - 사용한 Feature는?
        - 사용한 파라미터는?
        - 그 결과는?
    - 다양한 실험을 빠르게 진행하며, 손으로 기록하지 않고 자동으로 기록될 수 있도록 도와줄 도구가 필요
- Scratch로 구현하면?
    - Logger를 정의해서 필요할 때마다 Logger에 Feature, 파라미터 등을 저장
    - 값을 보기 위해 Logger를 Parsing
- Sacred는 위 방법을 데코레이터로 쉽게 사용할 수 있도록 도와줌
    - MLOps와 관련되는 부분으로, 국내의 다양한 머신러닝 강의에서 이런 부분을 다루지 않아 추가!
    
    <br>
- Sacred의 Main mechanisms
    - ConfigScopes : 함수의 local 변수를 편리하게 다룰 수 있음 @ex.config 데코레이터로 사용
    - Config Injection : 모든 함수에 있는 설정을 접근할 수 있음
    - Command-line interface : 커맨드 라인으로 파라미터를 바꿔서 실행할 수 있음
    - Observers : 실험의 모든 정보를 Observers에게 제공해 저장. MongoDB / S3 등 => 이번 강의에선 그냥 로컬에 저장
    - Automatic seeding : 실험의 무작위를 컨트롤할 때 도와줌

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import LinearRegression
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
from ipywidgets import interact
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import os
from numpy.random import permutation
from sklearn import svm, datasets
from sacred import Experiment
from sacred.observers import FileStorageObserver

plt.style.use('ggplot')
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

PROJECT_ID='predictfornyctaxi'

In [2]:
import numpy as np

### Scared 공식 홈페이지 예시(다른 사람이 작성한 코드)

In [56]:
# Notebook에선 interactive=True 지정해줘야 하고, 스크립트 형식이라면 이 옵션이 필요 없음
ex = Experiment('iris_rbf_svm', interactive=True)

# Decorator로 config 저장
@ex.config
def cfg():
    C = 1.0
    gamma = 0.7

# ex.main을 지정 => 이 때 cfg에 있는 인자들이 자동으로 injection됨
# 또한 Notebook에선 ex.main을 쓰고 스크립트 파일에선 ex.automain을 사용
@ex.main
def run(C, gamma):
    iris = datasets.load_iris()
    per = permutation(iris.target.size)
    iris.data = iris.data[per]
    iris.target = iris.target[per]
    clf = svm.SVC(C, 'rbf', gamma=gamma)
    clf.fit(iris.data[:90],
          iris.target[:90])
    return clf.score(iris.data[90:],
                   iris.target[90:])

In [57]:
run_result = ex.run()

WARNING - iris_rbf_svm - No observers have been added to this run
INFO - iris_rbf_svm - Running command 'run'
INFO - iris_rbf_svm - Started
INFO - iris_rbf_svm - Result: 0.9833333333333333
INFO - iris_rbf_svm - Completed after 0:00:00


In [58]:
run_result.config

{'C': 1.0, 'gamma': 0.7, 'seed': 717118411}

In [59]:
run_result.result

0.9833333333333333

In [60]:
run_result.experiment_info

{'name': 'iris_rbf_svm',
 'base_dir': '/Users/hyunsoolee/Documents/GitHub/TIL/Day-24',
 'sources': [],
 'dependencies': ['ipython==7.19.0',
  'ipywidgets==7.5.1',
  'matplotlib==3.3.2',
  'numpy==1.19.2',
  'pandas==1.1.3',
  'sacred==0.8.2',
  'scikit-learn==0.23.2',
  'seaborn==0.11.0'],
 'repositories': [],
 'mainfile': None}

### 기존 프로젝트에 통합

In [61]:
ex = Experiment('nyc-demand-prediction', interactive=True)

# experiment_dir가 없으면 폴더 생성하고 FileStorageObserver로 저장
experiment_dir = os.path.join('./', 'experiments')
if not os.path.isdir(experiment_dir): 
    os.makedirs(experiment_dir)
ex.observers.append(FileStorageObserver.create(experiment_dir))

### 전처리

In [62]:
%%time
base_query = """
WITH base_data AS 
(
  SELECT nyc_taxi.*, gis.* EXCEPT (zip_code_geom)
  FROM (
    SELECT *
    FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
    WHERE 
        EXTRACT(MONTH from pickup_datetime) = 1
        and pickup_latitude  <= 90 and pickup_latitude >= -90
    ) AS nyc_taxi
  JOIN (
    SELECT zip_code, state_code, state_name, city, county, zip_code_geom
    FROM `bigquery-public-data.geo_us_boundaries.zip_codes`
    WHERE state_code='NY'
    ) AS gis 
  ON ST_CONTAINS(zip_code_geom, st_geogpoint(pickup_longitude, pickup_latitude))
)

SELECT 
    zip_code,
    DATETIME_TRUNC(pickup_datetime, hour) as pickup_hour,
    EXTRACT(MONTH FROM pickup_datetime) AS month,
    EXTRACT(DAY FROM pickup_datetime) AS day,
    CAST(format_datetime('%u', pickup_datetime) AS INT64) -1 AS weekday,
    EXTRACT(HOUR FROM pickup_datetime) AS hour,
    CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_datetime) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend,
    COUNT(*) AS cnt
FROM base_data 
GROUP BY zip_code, pickup_hour, month, day, weekday, hour, is_weekend
ORDER BY pickup_hour
"""

base_df = pd.read_gbq(query=base_query, dialect='standard', project_id=PROJECT_ID)

INFO - pandas_gbq.gbq - Total time taken 7.71 s.
Finished at 2021-03-25 16:16:06.


CPU times: user 1.64 s, sys: 123 ms, total: 1.77 s
Wall time: 7.71 s


### Feature Engineering

In [63]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(base_df[['zip_code']])
ohe_output = enc.transform(base_df[['zip_code']]).toarray()
ohe_df = pd.concat([base_df, pd.DataFrame(ohe_output, columns='zip_code_'+enc.categories_[0])], axis=1)
ohe_df['log_cnt'] = np.log10(ohe_df['cnt'])

In [64]:
def split_train_and_test(df, date):
    """
    Dataframe에서 train_df, test_df로 나눠주는 함수
    
    df : 시계열 데이터 프레임
    date : 기준점 날짜
    """
    train_df = df[df['pickup_hour'] < date]
    test_df = df[df['pickup_hour'] >= date]
    return train_df, test_df

### Train/Test 나누기

In [65]:
train_df, test_df = split_train_and_test(ohe_df, '2015-01-24')

- 사용하지 않을 컬럼 삭제

In [66]:
del train_df['zip_code']
del train_df['pickup_hour']
del test_df['zip_code']
del test_df['pickup_hour']

In [67]:
y_train_raw = train_df.pop('cnt')
y_train_log = train_df.pop('log_cnt')
y_test_raw = test_df.pop('cnt')
y_test_log = test_df.pop('log_cnt')

In [68]:
y_true = y_test_raw.values.copy()

In [69]:
x_train = train_df.copy()
x_test = test_df.copy()

In [70]:
def evaluation(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    score = pd.DataFrame([mape, mae, mse], index=['mape', 'mae', 'mse'], columns=['score']).T
    return score

### 실험 설정
- 위에서 ex = Experiment('nyc-demand-prediction', interactive=True)했는데, ex.config로 설정을 저장
- ex.capture는 해당 설정을 사용해 함수를 리턴
- ex.main은 실험이 실행될 때 진행할 내용을 담음

In [71]:
@ex.config
def config():
    fit_intercept=True
    normalize=False

In [72]:
@ex.capture
def get_model(fit_intercept, normalize):
    return LinearRegression(fit_intercept, normalize)

In [73]:
# _log과 _run은 별도로 정의하지 않아도 함수의 인자로 사용 가능
@ex.main
def run(_log, _run):
    lr_reg = get_model()
    lr_reg.fit(x_train, y_train_raw)
    pred = lr_reg.predict(x_test)
    # log File에 로그 저장
    _log.info("Predict End")
    score = evaluation(y_test_raw, pred)
    _run.log_scalar('model_name', lr_reg.__class__.__name__)
    
    # Metrics쪽에 저장하고 싶으면 아래처럼 사용
    _run.log_scalar('metrics', score)
    
    # Result쪽에 저장하고 싶으면 아래처럼 사용
    return score.to_dict()

In [74]:
experiment_result = ex.run()

INFO - nyc-demand-prediction - Running command 'run'
INFO - nyc-demand-prediction - Started run with ID "4"
INFO - run - Predict End
INFO - nyc-demand-prediction - Result: {'mape': {'score': 4379287.123290663}, 'mae': {'score': 44848.791441706235}, 'mse': {'score': 1907131268986.4849}}
INFO - nyc-demand-prediction - Completed after 0:00:01


In [75]:
experiment_result.config

{'fit_intercept': True, 'normalize': False, 'seed': 515113179}

### Experiment 확인하기 위한 Parser
- Experiment에서 log찍는 방식에 따라 사용할 함수가 다름
    - 1) _run.log_scalar에 metrics을 저장하는 경우 : 추천
    - 2) @ex.main의 함수에 결과를 return하는 경우

In [88]:
# 1) _run.log_scalar에 metrics을 저장하는 경우
def parsing_output(ex_id):
    with open(f'./experiments/{ex_id}/metrics.json') as json_file:
        json_data = json.load(json_file)
    with open(f'./experiments/{ex_id}/config.json') as config_file:
        config_data = json.load(config_file)
    
    output_df = pd.DataFrame(json_data['model_name']['values'], columns=['model_name'], index=['score'])
    output_df['experiment_num'] = ex_id
    output_df['config'] = str(config_data)
    metric_df = pd.DataFrame(json_data['metrics']['values'][0]['values'])
    
    output_df = pd.concat([output_df, metric_df], axis=1)
    output_df = output_df.round(2)
    return output_df

In [85]:
# 2) @ex.main의 함수에 결과를 return하는 경우
def parsing_output(ex_id):
    with open(f'./experiments/{ex_id}/run.json') as json_file:
        json_data = json.load(json_file)
    output = pd.DataFrame(json_data['result'])
    return output

In [86]:
parsing_output(3)

,mae,mape,mse
score,44848.791442,4.379287e+06,1.907131e+12
